In [1]:
import spacy
from spacy.language import Language
import re
import hjson

In [ ]:
nlp=spacy.load("es_core_news_lg")
with open("../assets/replacements.hjson", "r") as file:
    replacements=hjson.load(file)
@Language.component("replacer_component")
def replacer_component(doc):
    text=doc.text
    for k, v in replacements.items():
        pattern=r"\b" + re.escape(k) + r"\b"
        text=re.sub(pattern, v, text, flags=re.IGNORECASE)
    new_doc=nlp.make_doc(text)
    return new_doc


nlp.add_pipe("replacer_component", first=True)

<function __main__.replacer_component(doc)>

In [ ]:
query=(nlp("Cuantos clave del reporte se emiten a diario que esten validaciones para MX_BM_R04C_0430_00_22 ")).text
terminos=["derogados", "derogado", "o vigentes", "no vigente"]
subsecuencias=["validaciones", "validacion", "valida"]
for termino in terminos:
    if termino in query:
        query=query.replace(termino, "(VIGENTE=0)")
if "VIGENTE=0" not in query:
    query+=" (VIGENTE=1)"
for termino in subsecuencias:
    if termino in query:
        query=query.replace(" (VIGENTE=1)", "")
print(query)

Cuantos CLAVE_REP se emiten a 01-DIA que esten validaciones para MX_BM_R04C_0430_00_22 


In [2]:
text = """
Para obtener todos los reportes pertenecientes al grupo de Liquidez donde el reporte está activo (VIGENTE = 1), puedes usar la siguiente consulta SQL:

```sql
SELECT * FROM reportes WHERE CLAVE_GRUPO = 'LIQUIDEZ' AND VIGENTE = 1;
```

**Explicación:**

- **SELECT \***: Esta cláusula selecciona todas las columnas de la tabla.
- **FROM reportes**: Especifica el nombre de la tabla de donde obtener los datos.
- **WHERE CLAVE_GRUPO = 'LIQUIDEZ'**: Filtra las filas para incluir solo aquellas donde la columna `CLAVE_GRUPO` es exactamente 'LIQUIDEZ'.
- **AND VIGENTE = 1**: Filtra adicionalmente los resultados para incluir solo las filas donde la columna `VIGENTE` es 1, indicando que el reporte está activo.

Esta consulta devolverá todas las filas en la tabla `reportes` que pertenecen al grupo de Liquidez y están actualmente activas.

### Resultados de la Consulta:
1. **Reporte 0430 perteneciente a BM**
   - **CLAVE_ENTIDADREGULADA:** BM
   - **CLAVE_PERIODO:** 03-MENS

2. **Reporte 0430 perteneciente a FF**
   - **CLAVE_ENTIDADREGULADA:** FF
   - **CLAVE_PERIODO:** 03-MENS

3. **Reporte 0430 perteneciente a SOF**
   - **CLAVE_ENTIDADREGULADA:** SOF
   - **CLAVE_PERIODO:** 03-MENS

### Explicación:
**REPORTE '0430':** Se trata de un identificador específico del reporte.
**CLAVE_ENTIDADREGULADA:** Son las entidades reguladas a las que pertenecen los reportes, en este caso: BM, FF y SOF.
**CLAVE_PERIODO:** Es el período al que corresponde el reporte. En este caso, es '03-MENS', lo que probablemente indica un período mensual.
**VIGENTE = 1:** Indica que los reportes están activos y vigentes.

| CLAVE_REP                         | CLAVE_PERIODO    | TEST_1 | TEST_2 |
|-----------------------------------|------------------|--------|--------|
| MX_BM_R04C_0430_00_22             | 03-MENS          | TEST_1 | TEST_2 |
| MX_FF_R04C_0430_00_22             | 03-MENS          | TEST_1 | TEST_2 |
| MX_SOF_R04C_0430_00_22            | 03-MENS          | TEST_1 | TEST_2 |
"""

import re
from tabulate import tabulate

def format_header(match):
    content = match.group(2).strip()
    return f'<h>{content}</h>'

def format_bold(match):
    content = match.group(1)
    return '<b>' + content + '</b>'

def format_table(text):
    lines = text.split('\n')
    formatted_lines = []
    table_lines = []
    in_table = False
    
    for line in lines:
        # Detecta líneas de tabla (que comienzan con |)
        if line.strip().startswith('|'):
            if not in_table:
                in_table = True
            table_lines.append(line.strip())
        elif in_table:
            # Si ya no estamos en una tabla, procesamos las líneas acumuladas
            if table_lines:
                # Convertimos las líneas de la tabla a una estructura de datos
                table_data = []
                headers = []
                for i, table_line in enumerate(table_lines):
                    # Eliminar los | del inicio y final y dividir por |
                    cells = [cell.strip() for cell in table_line.strip('|').split('|')]
                    if i == 0:  # Primera línea son los encabezados
                        headers = cells
                    elif i > 1:  # Saltamos la línea separadora (|----|)
                        table_data.append(cells)
                
                # Usar tabulate para formatear la tabla
                formatted_table = tabulate(table_data, headers=headers, tablefmt="grid")
                # Envolver en bloque de código
                formatted_lines.append("```")
                formatted_lines.append(formatted_table)
                formatted_lines.append("```")
                table_lines = []
            in_table = False
            formatted_lines.append(line)
        else:
            formatted_lines.append(line)
    
    # Si terminamos con una tabla abierta
    if table_lines:
        table_data = []
        headers = []
        for i, table_line in enumerate(table_lines):
            cells = [cell.strip() for cell in table_line.strip('|').split('|')]
            if i == 0:
                headers = cells
            elif i > 1:
                table_data.append(cells)
        formatted_table = tabulate(table_data, headers=headers, tablefmt="grid")
        formatted_lines.append("```")
        formatted_lines.append(formatted_table)
        formatted_lines.append("```")
    
    return '\n'.join(formatted_lines)

pat_header = r'^(#{2,3})\s+(.+?)$'
pattern = r'(?<!\\)\*\*(.*?)(?<!\\)\*\*'

text_pre = text
# Primero formateamos las tablas
text = format_table(text)

# Luego procesamos el resto del formato
lines = text.split('\n')
formatted_lines = []
for line in lines:
    # Procesamos los encabezados
    line = line.replace("```", "`").replace("sql", "").replace("javascript", "").replace("python", "").replace("json", "").replace("java", "")
    line = re.sub(pat_header, format_header, line)
    # Procesamos las negritas
    line = re.sub(pattern, format_bold, line)
    formatted_lines.append(line)
formatted_text = '\n'.join(formatted_lines)

print("Texto original:")
print("\nTexto formateado:")
print(formatted_text)

Texto original:

Texto formateado:

Para obtener todos los reportes pertenecientes al grupo de Liquidez donde el reporte está activo (VIGENTE = 1), puedes usar la siguiente consulta SQL:

`
SELECT * FROM reportes WHERE CLAVE_GRUPO = 'LIQUIDEZ' AND VIGENTE = 1;
`

<b>Explicación:</b>

- <b>SELECT \*</b>: Esta cláusula selecciona todas las columnas de la tabla.
- <b>FROM reportes</b>: Especifica el nombre de la tabla de donde obtener los datos.
- <b>WHERE CLAVE_GRUPO = 'LIQUIDEZ'</b>: Filtra las filas para incluir solo aquellas donde la columna `CLAVE_GRUPO` es exactamente 'LIQUIDEZ'.
- <b>AND VIGENTE = 1</b>: Filtra adicionalmente los resultados para incluir solo las filas donde la columna `VIGENTE` es 1, indicando que el reporte está activo.

Esta consulta devolverá todas las filas en la tabla `reportes` que pertenecen al grupo de Liquidez y están actualmente activas.

<h>Resultados de la Consulta:</h>
1. <b>Reporte 0430 perteneciente a BM</b>
   - <b>CLAVE_ENTIDADREGULADA:</b> BM
   -